# BOTPT Data detide 

We detide seafloor pressure measurements taken at The Axial Seamount's Central Caldera (RS03CCAL)  

In [ ]:
%reset -f

In [ ]:
# Load map of Axial caldera showing locations of BOTPT instruments (red circles)
from IPython.display import Image
Image(url = "https://www.pmel.noaa.gov/eoi/rsn/Axial-2017-OOI-caldera-ed-sm.png")

In [ ]:
#Zoomed in map of Axial caldera
from IPython.display import Image
Image(url = "https://www.pmel.noaa.gov/eoi/rsn/Axial-2017-OOI-zoom-ed-sm.png")

### API Information Setup

In [ ]:
USERNAME = 'OOIAPI-J97520T1AYPUNI'
TOKEN =  'TEMP-TOKEN-YD01XSNDIO57MP'

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import pyplot
from pylab import rcParams
import pickle as pk
import gc
import requests 
import matplotlib.gridspec as gridspec
import netCDF4 as nc

#### For mad help with Pandas https://chrisalbon.com
#### For all sorts of great repos https://github.com/rabernat/research_computing.git

#### Read Tide Data

In [ ]:
tide_file = '/home/jovyan/data/tide_data/formatted_pred/pred_E.txt'
df_tides_E = pd.read_csv(tide_file, delim_whitespace=True, dtype = object)
df_tides_E['datetime']=df_tides_E['year'] + '-' + df_tides_E['month'] + '-' + df_tides_E['day'] + \
            'T' + df_tides_E['hour'] + ':' + df_tides_E['minute'] + ':' + df_tides_E['second']
#df_tides_E = pd.DataFrame(tides_E)
df_tides_E.index=pd.to_datetime(df_tides_E['datetime'].values)
#df_tides_E.index = df_tides_E['parsed_time']
del df_tides_E['year']
del df_tides_E['month']
del df_tides_E['day']
del df_tides_E['hour']
del df_tides_E['minute']
del df_tides_E['second']
del df_tides_E['datetime']
df_tides_E['height'] = df_tides_E['height'].astype(float)

# columnsTitles=["datetime","height"]
# df_tides_E=df_tides_E.reindex(columns=columnsTitles)
df_tides_E.head()

#### Read Downsampled BOTPT Data

In [ ]:
#df_tides_E.height.plot()
# df_tides_E.sel(date=slice('2015-10-01', '2016-10-01')).plot()
# df_tides_E['5/3/2014':'5/4/2015'].plot()

In [ ]:
time = list(df_tides_E.index.values)
height = list(df_tides_E['height'].values)

time_int = []
time = list(pd.to_datetime(df_tides_E.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
plt.close()
fig, ax = plt.subplots()
fig.set_size_inches(28, 6)
hb1 = ax.hexbin(time_int, height, vmin=0, vmax=30, gridsize=(1500,100), mincnt=1, cmap='Greens', linewidths=0)
fig.colorbar(hb1, pad = 0.01)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_xlim(datetime.datetime(2014, 10, 1, 0, 0),datetime.datetime(2017, 12, 1, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()

### Instrument Information for MJ03F

In [ ]:
# Instrument Information RS03ASHS-MJ03B-09-BOTPTA304
subsite = 'RS03CCAL'
node = 'MJ03F'
sensor = '05-BOTPTA301'
method = 'streamed'
stream = 'botpt_nano_sample'
beginDT = '2015-01-01T00:00:00.000Z'
endDT = '2016-12-31T11:59:59.000Z'
base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'

data_request_url ='/'.join((base_url,subsite,node,sensor,method,stream))
params = {
    'beginDT':beginDT,
    'endDT':endDT,
}

In [ ]:
# Grab the data
# r = requests.get(data_request_url, params=params, auth=(USERNAME, TOKEN))
# data = r.json()

In [ ]:
len(data)

### Pull in data MJ03F

In [ ]:
MJ03F_time = []
MJ03F_pressure = []
for i in range(len(data)):
    MJ03F_time.append(nc.num2date(data[i]['time'],'seconds since 1900-01-01').replace(microsecond=0))
    MJ03F_pressure.append(data[i]['bottom_pressure'])

In [ ]:
MJ03F_time[1:3]

### Take both lists and make Pandas Data Frame

In [ ]:
d = {'time':MJ03F_time, 'pressure': MJ03F_pressure}
df =pd.DataFrame(d)
df.tail(3)

### Make preliminary PSI plot MJ03F 

In [ ]:
df.time = pd.DatetimeIndex(df.time)

In [ ]:
df.time

In [ ]:
plt.close()
fig, ax = plt.subplots()
ax.plot(MJ03F_time, MJ03F_pressure, marker=".", markersize=1, linestyle=None)
fig.set_size_inches(28, 6)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_ylabel('psia')
plt.xlabel('Time')
plt.xticks(rotation=30)
plt.tight_layout()
# fig.subplots_adjust(hspace=0)
# plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()

### Psi to Depth MJ03F
##### google "how to multiply a list by a scalar python..."

In [ ]:
MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]

### Make preliminary Depth plot MJ03F

In [ ]:
plt.close()
fig, ax = plt.subplots()
ax.plot(MJ03F_time, MJ03F_cal_depths, marker=".", markersize=1, linestyle=None)
fig.set_size_inches(28, 6)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_ylabel('meters')
plt.xlabel('Time')
plt.xticks(rotation=30)
plt.tight_layout()
# fig.subplots_adjust(hspace=0)
# plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()

### Instrument Information for MJ03E

In [ ]:
# Instrument Information RS03ASHS-MJ03B-09-BOTPTA304
subsite = 'RS03ECAL'
node = 'MJ03E'
sensor = '06-BOTPTA302'

data_request_url ='/'.join((base_url,subsite,node,sensor,method,stream))
params = {
    'beginDT':beginDT,
    'endDT':endDT,
    'limit':20000,   #Investigate!!! What are the other possible parameters? What does this one do?? 
}

In [ ]:
del r
del data
# Grab the data
r = requests.get(data_request_url, params=params, auth=(USERNAME, TOKEN))
data = r.json()

### Pull in data MJ03E

In [ ]:
MJ03E_time = []
MJ03E_pressure = []
for i in range(len(data)):
    MJ03E_time.append(nc.num2date(data[i]['time'],'seconds since 1900-01-01').replace(microsecond=0))
    MJ03E_pressure.append(data[i]['bottom_pressure'])

### Make preliminary PSI plot MJ03E 

In [ ]:
plt.close()
fig, ax = plt.subplots()
ax.plot(MJ03E_time, MJ03E_pressure, marker=".", markersize=1, linestyle=None)
fig.set_size_inches(28, 6)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_ylabel('psia')
plt.xlabel('Time')
plt.xticks(rotation=30)
plt.tight_layout()
# fig.subplots_adjust(hspace=0)
# plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()

### Psi to Depth MJ03E

In [ ]:
MJ03E_cal_depths = [MJ03E_pressure * 0.0670 for MJ03E_pressure in MJ03E_pressure]

### Make preliminary Depth plot MJ03E

In [ ]:
plt.close()
fig, ax = plt.subplots()
ax.plot(MJ03E_time, MJ03E_cal_depths, marker=".", markersize=1, linestyle=None)
fig.set_size_inches(28, 6)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_ylabel('meters')
plt.xlabel('Time')
plt.xticks(rotation=30)
plt.tight_layout()
# fig.subplots_adjust(hspace=0)
# plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()

### Subtract depth data at MJ03F from MJ03E.  
I'm guessing you could probably just subtract the raw pressure (or depth) data (WITH tides) from the two sites and it would effectively do the same thing.  That way, you wouldn't need to do deal with the predicted tides.  Might be worth testing, if you aren't going to need the de-tided data from individual stations.  Could simplify what you are doing.

In [ ]:
print(len(MJ03E_cal_depths))
print(len(MJ03F_cal_depths))
MJ03E_time[:10]

In [ ]:
MJ03F_time[:10]

In [ ]:
MJ03F_time[-10:]

In [ ]:
MJ03E_time[-10:]

In [ ]:
# zip(*iterables)
# Make an iterator that aggregates elements from each of the iterables.
# Returns an iterator of tuples, where the i-th tuple contains the i-th element from each of the argument sequences
# or iterables. The iterator stops when the shortest input iterable is exhausted. With a single iterable argument, 
# it returns an iterator of 1-tuples. With no arguments, it returns an empty iterator.

diff_depths = [MJ03E_cal_depths - MJ03F_cal_depths for MJ03E_cal_depths, MJ03F_cal_depths in zip(MJ03E_cal_depths, MJ03F_cal_depths)]

In [ ]:
print(len(diff_depths))

### Make preliminary Difference Plot 

In [ ]:
plt.close()
fig, ax = plt.subplots()
ax.plot(MJ03F_time, diff_depths, marker=".", markersize=1, linestyle=None)
fig.set_size_inches(28, 6)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_ylabel('meters')
plt.xlabel('Time')
plt.xticks(rotation=30)
plt.tight_layout()
# fig.subplots_adjust(hspace=0)
# plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()